In [266]:
import numpy as np
import time
#from termcolor import colored
from scipy.integrate import solve_ivp
from scipy.linalg import norm
from numpy import pi as pi
import scipy
from scipy import sparse
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from numpy.random import choice, normal
from numpy import concatenate
from scipy.sparse import coo_matrix
from scipy.sparse import kron
import pandas as pd
from mpl_toolkits import mplot3d
from numba import jit
from scipy.linalg import expm
import numbers
from numpy import imag, conj, cos, sin
from numpy import sqrt, zeros, diag, random, real, array, complex_, transpose, conjugate, exp, matmul, real, dot, add, sum
import pandas as pd
from numpy import dot
from scipy.sparse import bsr_matrix

df=pd.read_csv('Atom_positions__MACE_2_level_2000_atoms_Lax_10_3D_gas_fixed_size_k_along_x_eL_along_z_rabi_0,1.txt') 


t0=time.time()


Gamma_r=1    #  2*pi*6e6  #reference
Omega=2*Gamma_r   #Gamma_r  #Driving frequency , What are the units?




N_atom=200      #Number of atoms






lambda_r  =1    # in units
Lambda_las=1     # in units    Gamma_r*2*pi/(wL/c_light)   #Laser wavelenght  , What are the units?
e_L=array([0,0,1]/sqrt(1))

Theta1=pi/2
Theta2=pi/2   #they got [0,1,0]

e_0=array([0,0,1]/sqrt(1))   #array([sin(Theta1)*cos(Theta2),sin(Theta1)*sin(Theta2),cos(Theta1)])

K_laser=(2*pi/Lambda_las)*array([1,0,0])
k0=2*pi/Lambda_las
Gamma_dec=Gamma_r

# Clebsch-Gordan coefficient

# Clebsch-Gordan coefficient
C00=1    

# Positions


## Oblate Variances matrix: Set the shape of the atoms cloud

Sigma_x=20 *lambda_r/2    
Sigma_y=0.2*lambda_r/2 
Sigma_z=Sigma_y     /2


Oblate_variances=zeros((3,3))

Oblate_variances[0][0]=Sigma_x**2
Oblate_variances[1][1]=Sigma_y**2
Oblate_variances[2][2]=Sigma_z**2

Miu_pos=[0 , 0, 0]

Pos_atoms=(random.multivariate_normal(Miu_pos, Oblate_variances, N_atom))


Pos_atoms=Pos_atoms[Pos_atoms[:,0].argsort()]


for j in range(N_atom):
    Pos_atoms[j]=np.array([0.1*Lambda_las*j,0,0])
  
# 2*np.pi

#Pos_atoms
#Pos_atoms    



# Construct $I_{Lk}$ and $R_{Lk}$
######  Pos_atoms=array(df)


# Construct I_Lk and R_Lk

eL_e0c=dot(e_L,conj(e_0))

conj_0=conj(e_0)

DotKL_rk=[]   #summarize k_L.r_k for all k

for j in range(N_atom):
    DotKL_rk.append(dot(Pos_atoms[j],K_laser))

DotKL_rk_exp=exp(1j*array(DotKL_rk))


ZL_k= DotKL_rk_exp*eL_e0c


ImL_k=imag(ZL_k)
RmL_k=real(ZL_k)


# Construct $G_{ij}$

# Construct the matrix R00
#t0=time.time()

@jit
def fun_Pkl0(k,l,k0=k0):
    
    VecR_kl=Pos_atoms[k]-Pos_atoms[l]
    R_kl=sqrt(dot(VecR_kl,VecR_kl))
    #R_kl=norm(VecR_kl)
    
    
    #Pkl0=dot(conj(e_0),VecR_kl)
    Pkl0=dot(conj_0,VecR_kl)/R_kl
    Pkl_0nC=(abs(Pkl0))**2
    
    M0kl=k0*R_kl
    
    return (1-Pkl_0nC)/M0kl+(1-3*Pkl_0nC)*(1j/(M0kl)**2-1/(M0kl)**3)
    


MatL=zeros((N_atom,N_atom),dtype='complex64')
MatR=zeros((N_atom,N_atom),dtype='complex64')


for i in range(N_atom):
    for j in range(i+1,N_atom):

        VecR_ij=Pos_atoms[i] -Pos_atoms[j]

        MatL[i][j]=exp(1j*k0*sqrt(dot(VecR_ij,VecR_ij)))
        MatR[i][j]=fun_Pkl0(i,j,k0)

                
MatR=MatR+MatR.T
MatL=MatL+(MatL.T)


Gij=(3*Gamma_dec/4)*MatL*MatR


#t1=time.time()

Gij=Gij + 1j*0.5*Gamma_r*np.identity(N_atom) 

Abs_Gij=abs(Gij)

ReG=real(Gij)
ImG=imag(Gij)

#tspan=np.linspace(0,7,250)
Collect    =0



#t0=time.time()



In [337]:
Atom_cutoff=5

In [338]:
Sigma_eg=csr_matrix(np.array([[0,1],
                              [0,0]]),dtype="complex64")
Id_2LS  =csr_matrix(np.array([[1,0],
                              [0,1]]),dtype="complex64")

Zero_2LS=csr_matrix(np.array([[0,0],
                              [0,0]]),dtype="complex64")

state   =csr_matrix(np.array([[0,0],
                              [0,1]]),dtype="complex64")


Sigma_ee=csr_matrix(np.array([[1,0],
                              [0,0]]),dtype="complex64")

Sigma_egT=Sigma_eg.T    




def index_Gij(arr,Atom_cutoff):
    
    Ordered_N = list(np.argsort(arr)[::-1][:Atom_cutoff])
    
    return Ordered_N

Index_N=[]

Rem_iden=abs(Gij - 1j*0.5*Gamma_r*np.identity(N_atom))


for j in range(Rem_iden.shape[0]):
    
    test_list = list(Rem_iden[j])
    
    Index_N.append( [j]+list(index_Gij(test_list,Atom_cutoff-1))  )



#@jit
def tensor_id(ArrayL ):
    
    Tensor_id=kron(ArrayL[0],ArrayL[1])
    
    Array_id=Atom_cutoff*[Id_2LS]    
    
    for j in range(2,len(ArrayL)):
        
        Tensor_id=kron(Tensor_id,Array_id[j])
        
    
    Ten_s=csr_matrix(Tensor_id)
    
    csr_matrix.eliminate_zeros(Ten_s)  #remove zeros!!!
    
    return coo_matrix(Ten_s)



#@jit
def tensor_kj( k , j):   #tensor among k,j are Positions in the cluster after the G classification
                         #k is +sigma and j is -sigma 
    Array_id=Atom_cutoff*[Id_2LS]  
    
    Array_id[k]=Sigma_eg
    Array_id[j]=Sigma_egT
    
    Tensor_kj=kron(Array_id[0],Array_id[1])
    
    for j in range(2,len(Array_id)):
        
        Tensor_kj=kron(Tensor_kj,Array_id[j])
        
        csr_matrix.eliminate_zeros(csr_matrix(Tensor_kj))
        
        
            
    Ten_kj=csr_matrix( Tensor_kj ) 
        
    csr_matrix.eliminate_zeros(Ten_kj)
        
        
    
    return coo_matrix(Ten_kj)



#@jit
def tensor_k_ee( k  ):  #Excited state k is a Position in the cluster after the G classification
    
    Array_id=Atom_cutoff*[Id_2LS]  
    
    Array_id[k]=Sigma_ee
    
    Tensor_k    =kron(Array_id[0],Array_id[1])
    
    for j in range(2,len(Array_id)):
        
        Tensor_k=kron(Tensor_k,Array_id[j])
        
    
    
    Tensor_k=csr_matrix( Tensor_k ) 
    
    csr_matrix.eliminate_zeros(Tensor_k)
        
    
    return coo_matrix(Tensor_k)



#@jit
def tensor_k( k  ):  #sigmaplus k
    
    Array_id=Atom_cutoff*[Id_2LS]  
    
    Array_id[k]=Sigma_eg
    
    Tensor_k    =kron(Array_id[0],Array_id[1])
    
    for j in range(2,len(Array_id)):
        
        Tensor_k=kron(Tensor_k,Array_id[j])
        
    
    Tensor_k=csr_matrix( Tensor_k ) 
    
    csr_matrix.eliminate_zeros(Tensor_k)
        
    
    return coo_matrix(Tensor_k)



#@jit
def init_rho(Atom_cutoff=Atom_cutoff,state=state):
    
    state_fact=Atom_cutoff*[state]
    
    Rho_in=kron(state_fact[0],state_fact[1])
    
    for j in range(2,Atom_cutoff):
        
        Rho_in=kron(Rho_in,state_fact[j])
        
        
    Rho_in=csr_matrix( Rho_in ) 
    
    csr_matrix.eliminate_zeros(Rho_in)
    
    
        
    return coo_matrix(Rho_in)





#@jit
def Hamil_atom(index_center):
    
    Index_list=Index_N[index_center]
    Zero_gen=tensor_id(Atom_cutoff*[ Zero_2LS ])
    Z2=Zero_gen
    
    for k in range(Atom_cutoff):
        
        IL_k = Index_list[k]
        
        #print(IL_k)
        
        zL_k = ZL_k[IL_k]
        Ten_k= tensor_k(k)
        
        Z2   = Z2 -(Omega)*(zL_k*Ten_k  +conj(zL_k)*(Ten_k.T) )
        
    return Z2

Sigma_EGalt=(tensor_k(0)).reshape(1,4**Atom_cutoff)
Sigma_EEalt=(tensor_k_ee(0)).reshape(1,4**Atom_cutoff)
Sigma_Min_alt=(tensor_k(0).T).reshape(1,4**Atom_cutoff)

In [339]:
Ivec=[4**Atom_cutoff-1]

Jvec=[0]
Vvec=[1]


Init_rho=(sparse.coo_matrix((Vvec,(Ivec,Jvec)),shape=(4**Atom_cutoff,1))) 


    #@jit
def f(t, yR,M_vect,b):
 
    MatA=M_vect@yR
    return MatA



def Hamil_effec(i_index, Expect_minus):   #this is just for the site labelled by index_center
    
    Vect_Gi=Gij[i_index]
    
    All_Ind_effective_drive=dot(Vect_Gi,Expect_minus.todense())[0,0]
    
    return All_Ind_effective_drive

#Hamil_effec(0, Sigma_m_rev[:,0])




def GdotMinus(Expect_minus):
    
    return (Expect_minus.T).dot(Gij)




Rho_t= sparse.lil_matrix((4**Atom_cutoff, N_atom),dtype="complex64")   #saves the density matrices for a given time t
for j in range(N_atom):
    Rho_t[:,j]=Init_rho


        
tfinal=6
Lenght=80
tspan=np.linspace(0,tfinal,Lenght)    #Here the time definition is important because it will tell you how to divide the elements



Sigma_m_rev=  sparse.lil_matrix((N_atom, len(tspan)),dtype="complex64")
Sigma_ee_rev= sparse.lil_matrix((N_atom, len(tspan)),dtype="complex64")



for j in range(N_atom):
    
    Sigma_m_rev[j,0] =Sigma_Min_alt.dot( Rho_t[:,j])[0,0]
    Sigma_ee_rev[j,0]=Sigma_EEalt.dot( Rho_t[:,j])[0,0]
    

#Sigma_m_rev[:,0][1,0]




def hamil_clust(index_center,ind_ref,Expect_minus):
    
    
    Index_list=Index_N[index_center]
    
    ind_ref=Index_list[ind_ref]
    
    Driv_clust=0
    
    for ind_list in Index_list:
        
        Driv_clust=Driv_clust+Gij[ind_ref, ind_list ]*Expect_minus[ind_list , 0 ]
    
    return Driv_clust
        

#Gmin[Index_list[j],0] 



def Effec_Hamil( index_center ,tind_):
    
    Expect_minus=Sigma_m_rev[:,tind_]
    
    Index_list=Index_N[index_center]
    
    Gmin=GdotMinus(Sigma_m_rev[:,tind_])
    
    H_effect = 0*tensor_k(0)
    
    for k in range(Atom_cutoff):
        H_effect = H_effect -(Gmin[0,Index_list[k]]  -  hamil_clust(index_center , k  ,Expect_minus)  ) * tensor_k(k)
    
    return (H_effect+conj(H_effect.T))





In [340]:
def Part_H_L_R_LR(index_center):
    
    Array_id=Atom_cutoff*[Id_2LS] 
    Index_list=Index_N[index_center]
    Zero_gen=tensor_id(Atom_cutoff*[ Zero_2LS ])
    
    Z1=Zero_gen
    Z2=Zero_gen
    Z3=kron(Zero_gen,Zero_gen)
    
    
    Col_Decay=kron(Zero_gen,Zero_gen)
    
    Identity=tensor_id( Atom_cutoff*[Id_2LS])
    
    Col=Collect
    
    D_R = Zero_gen
    D_L = Zero_gen
    D_RL= kron(Zero_gen,Zero_gen)

    Hamil=Zero_gen
    
    Splus=Zero_gen
    
    
    Hamil_at=Hamil_atom(index_center)   
    Hamil_vect= -1j *(kron(Identity,  Hamil_at )-kron(Hamil_at.T,Identity))
    
    for k in range(Atom_cutoff):
        
        IL_k = Index_list[k]

        zL_k = ZL_k[IL_k]

        Ten_k= tensor_k(k)
        
        Ten_k_ee=tensor_k_ee(k)
        
        
        Splus=Splus+Ten_k
        
        Imkk=ImG[k,k]  
        
        
            #Z2   = Z2+ 1j*(Omega)*(zL_k*Ten_k  +conj(zL_k)*(Ten_k.T) )
        
        
        D_R   = D_R  - ( Imkk       )*Ten_k_ee
        D_L   = D_L  - ( Imkk       )*Ten_k_ee
        D_RL  = D_RL + ( 2 * Imkk    )*kron( Ten_k.T , Ten_k.T  )

        for j in range(k+1,Atom_cutoff):
            
            IL_j  =Index_list[j]

            Ten_kj= tensor_kj( k , j )
            
            ReGkj=ReG[IL_k,IL_j] 
            ImGkj=ImG[IL_k,IL_j]
            ReGjk=ReGkj   
            ImGjk=ImGkj
            
            
            Z1=Z1+ (1j*ReGkj-ImGkj)*(Ten_kj+ Ten_kj.T  )
            
            Ten_j=tensor_k(j)
            
            
            Tenk_Tenj=kron(Ten_k.T,Ten_j.T)

            Tenj_Tenk=kron(Ten_j.T,Ten_k.T)
            
            
            Z3=Z3+2*( ImGkj*Tenk_Tenj   +  ImGjk* Tenj_Tenk )

            #print( tensor_kj( k , j ).todense(), ReG[IL_k,IL_j],(IL_k,IL_j)  )
            
            #print(k,j)



            Hamil=Hamil+(1j*ReGkj)*(Ten_kj+ Ten_kj.T)

    Hamil=Hamil+Hamil_at                 
        
    L_R = Z2+Z1
    L_L =-(Z2.T)+conj(Z1.T) 
    L_RL=Z3
    

    #print(L_R.shape, L_L.shape, L_RL.shape)
    
    Vectorization=  ( kron(Identity,  L_R+D_R )+kron(L_L+D_L,Identity )+L_RL+D_RL ) #   + Col * Col_Decay  )

    
    return  Vectorization+Hamil_vect  #, Hamil #cl_sparse(Vectorization) #clean_sparse(Vectorization)


In [341]:
def Part_time_dep(index_center,tind_):

    Identity=tensor_id( Atom_cutoff*[Id_2LS])
    
    Hamil_at  =  Effec_Hamil( index_center ,tind_)  #Put zero to ignore mean-field 
    Hamil_vect= -1j *(kron(Identity,  Hamil_at )-kron(Hamil_at.T,Identity))
        
    return  Hamil_vect  #, Hamil #cl_sparse(Vectorization) #clean_sparse(Vectorization)


In [342]:
ty=time.time()


Time_ind_hamil=[]   #Calculates all the time-independent Lindblandian vectorized

for cent_ind in range(N_atom):
    
    Time_ind_hamil.append(Part_H_L_R_LR(cent_ind))
    
    #print(cent_ind*100/N_atom)
    
    
tv=time.time()

print(tv-ty)

34.85497546195984


In [343]:


def rho_part(Rho_mt,inst_time):
    
    Result_fin=Rho_mt

    for cent_ind in range(N_atom):
        
        #Mat_dyn=Time_ind_hamil[cent_ind]+Part_time_dep(cent_ind,inst_time)
        
        Mat_dyn=Time_ind_hamil[cent_ind]+Part_time_dep(cent_ind,inst_time)
        
        sol_cent_ind = solve_ivp(f,[tspan[inst_time],tspan[inst_time+1]],list(array((Rho_mt[:,cent_ind].todense()).T,dtype="complex")[0])
                                 ,args=(Mat_dyn,None),dense_output=False , t_eval= [tspan[inst_time+1]])
        
        Result_fin[:,cent_ind]=sol_cent_ind.y
        
        #print(cent_ind)
        
    return Result_fin
        

In [344]:
Rho0= sparse.lil_matrix((4**Atom_cutoff, N_atom),dtype="complex64")   #saves the density matrices for a given time t
for j in range(N_atom):
    Rho0[:,j]=Init_rho

In [ ]:
for tind_ in range(len(tspan)-1):
    
    Rho0=rho_part(Rho0,tind_)
    
    
    for j in range(N_atom):
    
        Sigma_m_rev[j ,tind_+1] = Sigma_Min_alt.dot( Rho0[:,j] )[0,0]
        Sigma_ee_rev[j,tind_+1] = Sigma_EEalt.dot(   Rho0[:,j] )[0,0]
        
    print(str((tind_+1)*100/len(tspan)))
    

1.25
2.5
3.75


In [ ]:
Tot_minus=[]
Tot_excit=[]

for j in range(len(tspan)):
    
    Tot_minus.append( sum(Sigma_m_rev[:,j]) )
    Tot_excit.append( sum(Sigma_ee_rev[:,j]) )
    

In [ ]:
#Texc=real(Tot_excit)
#Tx=real(Tot_minus)
#Ty=imag(Tot_minus)



In [ ]:
Smint = np.array(Tot_minus)
Seet  = real(np.array(Tot_excit))
Time_array=np.array(tspan)

In [ ]:

# Displaying the array


file = open("Results"+"_Cluster_size_"+str(Atom_cutoff)+"_Omega_"+str(Omega)+"_Lax_"+str(Sigma_x)+"_Tspan_"+str(tspan[-1])+".txt", "w+")

# Saving the array in a text file

char1="N_atom="+str(N_atom)
char2="Cluster_size="+str(Atom_cutoff)
char3="Omega/Gamma="+str(Omega)



file.write(char1)
file.write('\n')

file.write(char2)
file.write('\n')

file.write(char3)
file.write('\n')


file.write('\n')

content1 = "tspan=np.array("+str(Time_array)+")"
file.write(content1)
file.write('\n')
file.write('\n')

content2 = "Seet=np.array("+ str(Seet)+")"
file.write(content2)
file.write('\n')
file.write('\n')

content12= "Sminust==np.array("+ str(Smint)+")"
file.write(content12)
file.write('\n')
file.write('\n')

file.close()

## Displaying the contents of the text file
#file = open("Array_MACE.txt", "r")
#content = file.read()

#print("Array contents in sample.txt: ", content)
#file.close()

t1=time.time()


In [ ]:
Atom_cutoff

In [ ]:
#Tot_excit2=Tot_excit
#Tot_minus2=Tot_minus

#Tot_excit3=Tot_excit
#Tot_minus3=Tot_minus

#Tot_excit4=Tot_excit
#Tot_minus4=Tot_minus

Tot_excit5=Tot_excit
Tot_minus5=Tot_minus

#Tot_excit6=Tot_excit
#Tot_minus6=Tot_minus

#Tot_excit7=Tot_excit
#Tot_minus7=Tot_minus

#Tot_excit8=Tot_excit
#Tot_minus8=Tot_minus

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
#from google.colab import files

"""In order to provide a better presentation of the graphs we use the rcParams options shown below."""

import matplotlib
matplotlib.rcParams['text.usetex'] = False
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams['figure.figsize'] = (11.0, 8.0)

from numpy import cos, sin

label_size = 20

from numpy import sqrt

import time
from numba import jit
from numpy import sin, cos

T0=time.time()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(tspan,real(Tot_excit2),label="CMF 2")
plt.plot(tspan,real(Tot_excit3),label="CMF 3")
plt.plot(tspan,real(Tot_excit4),label="CMF 4")
plt.plot(tspan,real(Tot_excit5),label="CMF 5")
#plt.plot(tspan,real(Tot_excit6),label="CMF 6")
#plt.plot(tspan,real(Tot_excit7),label="CMF 7")



#plt.plot(tspan,Texc,label="CMF")
#plt.plot(tspan,real(Tot_excit1),label="MACE")

plt.legend()

In [ ]:
plt.plot(tspan,real(Tot_minus2),"--",label="CMF 2 x")
plt.plot(tspan,imag(Tot_minus2),label="CMF 2 y")

plt.plot(tspan,real(Tot_minus3),"--",label="CMF 3 x")
plt.plot(tspan,imag(Tot_minus3),label="CMF 3 y")

plt.plot(tspan,real(Tot_minus4),"--",label="CMF 4 x")
plt.plot(tspan,imag(Tot_minus4),label="CMF 4 y")

plt.plot(tspan,real(Tot_minus5),label="CMF 5 x")
plt.plot(tspan,imag(Tot_minus5),label="CMF 5 y")

#plt.plot(tspan,real(Tot_minus6),label="CMF 6 x")
#plt.plot(tspan,imag(Tot_minus6),label="CMF 6 y")

#plt.plot(tspan,real(Tot_minus7),label="CMF 7 x")
#plt.plot(tspan,imag(Tot_minus7),label="CMF 7 y")

#plt.plot(tspan,Tx,"--",label="CMF x")
#plt.plot(tspan,Ty,"--",label="CMF y")

plt.legend()